# CDV position snapshot (single frame)

Build a `TrajectorySet` from CSV (t column treated as frame index), grab x/y coordinates at a specific time `t0` across all trajectories, assemble a displacement table over `delta_t` frames, and visualize a scatter of positions.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from trajkit import Trajectory, TrajectorySet

csv_path = "/Users/mehdi/particleTraj/res_xyti_time11.csv"  # update to your path
t0 = 0.0        # time of interest (frames, since we treat t as frame index)
time_tol = 1e-3 # tolerance (frames) when matching t0 inside each trajectory


def normalize_track_id(raw_id):
    """Convert numeric ids like 1.0 -> "1" while keeping other labels intact."""
    try:
        as_int = int(raw_id)
        if float(raw_id) == as_int:
            return str(as_int)
    except Exception:
        pass
    return str(raw_id)


# Build TrajectorySet
df = pd.read_csv(csv_path).sort_values(["id", "t"])
ts = TrajectorySet(
    dataset_id="airwater_1um",
    units={"t": "frame", "x": "pixel"},
    meta={"source": str(csv_path)},
)

for tid, df_tid in df.groupby("id", sort=False):
    df_tid = df_tid.sort_values("t").reset_index(drop=True)
    coords = df_tid[["x", "y"]].to_numpy(dtype=float)
    frames = df_tid["t"].to_numpy(dtype=float)  # treat 't' column as frame index
    ts.add(
        Trajectory(
            track_id=normalize_track_id(tid),
            x=coords,
            frame=frames,
            frame_rate_hz=1.0,  # so time_seconds() == frame index
        )
    )

ts.summary_table().head()

In [ ]:
# Build displacement table across all trajectories (delta_t in frames)
delta_t_frames = 1.0
disp_rows = []
for tid, tr in ts.trajectories.items():
    t = tr.time_seconds()
    frames = tr.frame if tr.frame is not None else np.arange(len(t), dtype=int)
    targets = t + delta_t_frames
    idx = np.searchsorted(t, targets, side="left")
    for i, j in enumerate(idx):
        if j >= len(t):
            continue
        if abs(t[j] - targets[i]) > time_tol:
            continue
        dx = tr.x[j] - tr.x[i]
        row = {
            "track_id": tid,
            "t": float(t[i]),
            "frame": int(frames[i]) if frames is not None else i,
        }
        for k in range(tr.D):
            row[f"x{k}"] = float(tr.x[i, k])
            row[f"dx{k}"] = float(dx[k])
        disp_rows.append(row)

disp_df = pd.DataFrame(disp_rows)
print(f"Displacement rows: {len(disp_df)} (delta_t={delta_t_frames} frames)")
disp_df.head()


In [ ]:
# Build correlation batch using displacement table as both source and tracer
from trajkit.cdv import correlation_batch, distance_threshold_pair_filter

disp_sub = disp_df[disp_df["frame"] < 3]
position_cols = [c for c in disp_sub.columns if c.startswith("x") and not c.startswith("dx")]
motion_cols = [c for c in disp_sub.columns if c.startswith("dx")]
max_pair_distance = 500  # drop pairs separated by more than this distance

corr_batch , ensemble = correlation_batch(
    disp_sub,
    disp_sub,
    source_frame_col="frame",
    tracer_frame_col="frame",
    source_position_cols=position_cols,
    tracer_position_cols=position_cols,
    source_motion_cols=motion_cols,
    tracer_motion_cols=motion_cols,
    pair_filter=distance_threshold_pair_filter(max_pair_distance),
)
print(f"Correlation pairs: {corr_batch.n_pairs}")
corr_batch.meta.head()


In [ ]:
batch_r_df = batch_r.to_dataframe()
batch_r_df.head()
plt.figure(figsize=(6, 6))
plt.scatter(batch_r_df["rel_pos_0"], batch_r_df["rel_pos_1"], s=1)
plt.quiver(batch_r_df["rel_pos_0"], batch_r_df["rel_pos_1"], batch_r_df["tracer_motion_0"], batch_r_df["tracer_motion_1"], scale=70, color='r')
plt.show()

In [ ]:
# Define grid for correlation ensemble using meshgrid centers and accumulate by frame
import numpy as np
from trajkit.cdv import correlation_batch, CorrelationEnsembleAccumulator



x = np.linspace(-300.0, 300.0, 50)
y = np.linspace(-300.0, 300.0, 50)
X, Y = np.meshgrid(x, y, indexing='xy')
grid_centers = np.stack([X.ravel(), Y.ravel()], axis=1)

ensemble = CorrelationEnsembleAccumulator(
    grid_centers,
    kernel=50.0,  # hard cutoff radius in rel_pos space
    value_fn=lambda rel, tracer, source, meta_row: tracer,
    weight_fn=lambda rel, tracer, source, meta_row: np.linalg.norm(source),
)

# Add two batches (frames 1 and 2)
disp_f12 = disp_df[disp_df['frame'].isin([1, 2])]
batch_f1, _ = correlation_batch(
    disp_f12[disp_f12['frame'] == 1],
    disp_f12[disp_f12['frame'] == 1],
    source_frame_col='frame', tracer_frame_col='frame',
    source_position_cols=position_cols, tracer_position_cols=position_cols,
    source_motion_cols=motion_cols, tracer_motion_cols=motion_cols,
)
batch_f2, _ = correlation_batch(
    disp_f12[disp_f12['frame'] == 2],
    disp_f12[disp_f12['frame'] == 2],
    source_frame_col='frame', tracer_frame_col='frame',
    source_position_cols=position_cols, tracer_position_cols=position_cols,
    source_motion_cols=motion_cols, tracer_motion_cols=motion_cols,
)



In [ ]:
ensemble = CorrelationEnsembleAccumulator(
    grid_centers,
    kernel=50.0,  # hard cutoff radius in rel_pos space
    value_fn=lambda rel, tracer, source, meta_row: tracer,
    weight_fn=lambda rel, tracer, source, meta_row: np.linalg.norm(source),
)

In [ ]:
ensemble.add(batch_f1)

In [ ]:
ensemble.add(batch_f1)
ensemble.add(batch_f2)
mean, sum_w, counts = ensemble.finalize()
mean.shape, float(sum_w.sum()), float(counts.sum() if counts is not None else 0)

In [ ]:
import numpy as np
from trajkit.cdv import correlation_batch, CorrelationEnsembleAccumulator, distance_threshold_pair_filter

position_cols = [c for c in disp_df.columns if c.startswith("x") and not c.startswith("dx")]
motion_cols = [c for c in disp_df.columns if c.startswith("dx")]
max_pair_distance = 600  # drop pairs separated by more than this distance


x = np.linspace(-400.0, 400.0, 100)
y = np.linspace(-400.0, 400.0, 100)
X, Y = np.meshgrid(x, y, indexing='xy')
grid_centers = np.stack([X.ravel(), Y.ravel()], axis=1)

ensemble = CorrelationEnsembleAccumulator(
    grid_centers,
    kernel=30.0,  # hard cutoff radius in rel_pos space
    value_fn=lambda rel, tracer, source, meta_row: tracer,
    weight_fn=lambda rel, tracer, source, meta_row: np.linalg.norm(source),
)
for i in range(1000):
    disp_temp = disp_df[disp_df['frame'] == i]
    batch, _ = correlation_batch(
        disp_temp,
        disp_temp,
        source_frame_col='frame', tracer_frame_col='frame',
        source_position_cols=position_cols, tracer_position_cols=position_cols,
        source_motion_cols=motion_cols, tracer_motion_cols=motion_cols,
        pair_filter=distance_threshold_pair_filter(max_pair_distance),
    )
    batch_r = batch.rotate_to_source_x()
    ensemble.add(batch_r)
    print(f"{i}")
mean, sum_w, counts = ensemble.finalize()

In [ ]:
ff = ensemble.sum_v          # shape: (M, 2) for x/y components
weights = ensemble.sum_w     # shape: (M,)
W = weights.reshape(Y.shape)
U = ff[:, 0].reshape(Y.shape)/W
V = ff[:, 1].reshape(Y.shape)/W

mag = np.log10(np.sqrt(U**2 + V**2))
plt.figure(figsize=(8, 6))
# Contour of magnitude
cs = plt.contourf(X, Y, mag, levels=20, cmap="viridis")
plt.colorbar(cs, label="||ff||")

# Streamlines (linewidth scales with magnitude)
plt.streamplot(
    X, Y, U, V,
    color="k",
    density=1.2,
    arrowsize=1.2,
)

plt.xlabel("x")
plt.ylabel("y")
plt.title("Correlation ensemble field")
plt.axis("equal")
plt.show()

In [ ]:
import numpy as np
from trajkit.cdv import correlation_batch, CorrelationEnsembleAccumulator, distance_threshold_pair_filter
from time import perf_counter
from IPython.display import clear_output, display

position_cols = [c for c in disp_df.columns if c.startswith("x") and not c.startswith("dx")]
motion_cols = [c for c in disp_df.columns if c.startswith("dx")]
max_pair_distance = 600  # drop pairs separated by more than this distance


x = np.linspace(-400.0, 400.0, 100)
y = np.linspace(-400.0, 400.0, 100)
X, Y = np.meshgrid(x, y, indexing='xy')
grid_centers = np.stack([X.ravel(), Y.ravel()], axis=1)

ensemble2 = CorrelationEnsembleAccumulator(
    grid_centers,
    kernel=30.0,  # hard cutoff radius in rel_pos space
    value_fn=lambda rel, tracer, source, meta_row: tracer,
    weight_fn=lambda rel, tracer, source, meta_row: np.linalg.norm(source),
)
times = []
for i in range(200):
    t0 = perf_counter()
    disp_temp = disp_df[disp_df['frame'] == i]
    batch, _ = correlation_batch(
        disp_temp,
        disp_temp,
        source_frame_col='frame', tracer_frame_col='frame',
        source_position_cols=position_cols, tracer_position_cols=position_cols,
        source_motion_cols=motion_cols, tracer_motion_cols=motion_cols,
        pair_filter=distance_threshold_pair_filter(max_pair_distance),
    )
    batch_r = batch.rotate_to_source_x()
    ensemble2.add(batch_r)
    t1 = perf_counter()
    duration = t1 - t0
    times.append(duration)

    # update plot each iteration
    clear_output(wait=True)
    plt.figure(figsize=(6, 3))
    plt.plot(np.arange(1, len(times) + 1), times, '-o', ms=3)
    plt.xlabel("iteration")
    plt.ylabel("time (s)")
    plt.title("Per-iteration time")
    plt.grid(True)
    display(plt.gcf())
    plt.close()

    print(f"{i}  iter_time={duration:.4f}s")
mean, sum_w, counts = ensemble.finalize()

In [ ]:

ff = ensemble.sum_v          # shape: (M, 2) for x/y components
weights = ensemble.sum_w     # shape: (M,)

# Reshape to grid
U = ff[:, 0].reshape(Y.shape)


# Mask cells with no weight
mask = W <= 0
U_masked = np.ma.array(U, mask=mask)
V_masked = np.ma.array(V, mask=mask)
mag_masked = np.ma.array(mag, mask=mask)

plt.figure(figsize=(8, 6))
# Contour of magnitude
cs = plt.contourf(X, Y, mag_masked, levels=20, cmap="viridis")
plt.colorbar(cs, label="||ff||")

# Streamlines (linewidth scales with magnitude)
plt.streamplot(
    X, Y, U_masked, V_masked,
    color="k",
    density=1.2,
    linewidth=1 + 2 * (mag_masked / (mag_masked.max() + 1e-9)),
    arrowsize=1.2,
)

plt.xlabel("x")
plt.ylabel("y")
plt.title("Correlation ensemble field")
plt.axis("equal")
plt.show()


In [ ]:
import numpy as np
from trajkit.cdv import correlation_batch, CorrelationEnsembleAccumulator, distance_threshold_pair_filter
from time import perf_counter
from IPython.display import clear_output, display

position_cols = [c for c in disp_df.columns if c.startswith("x") and not c.startswith("dx")]
motion_cols = [c for c in disp_df.columns if c.startswith("dx")]
max_pair_distance = 600  # drop pairs separated by more than this distance


x = np.linspace(-400.0, 400.0, 100)
y = np.linspace(-400.0, 400.0, 100)
X, Y = np.meshgrid(x, y, indexing='xy')
grid_centers = np.stack([X.ravel(), Y.ravel()], axis=1)

ensemble2 = CorrelationEnsembleAccumulator(
    grid_centers,
    kernel=30.0,  # hard cutoff radius in rel_pos space
    value_fn=lambda rel, tracer, source, meta_row: tracer,
    weight_fn=lambda rel, tracer, source, meta_row: np.linalg.norm(source),
)
times = []

i = 1
t0 = perf_counter()
disp_temp = disp_df[disp_df['frame'] == i]
batch, _ = correlation_batch(
    disp_temp,
    disp_temp,
    source_frame_col='frame', tracer_frame_col='frame',
    source_position_cols=position_cols, tracer_position_cols=position_cols,
    source_motion_cols=motion_cols, tracer_motion_cols=motion_cols,
    pair_filter=distance_threshold_pair_filter(max_pair_distance),
)
batch_r = batch.rotate_to_source_x()
ensemble2.add(batch_r)
t1 = perf_counter()
duration = t1 - t0
times.append(duration)

# update plot each iteration
clear_output(wait=True)
plt.figure(figsize=(6, 3))
plt.plot(np.arange(1, len(times) + 1), times, '-o', ms=3)
plt.xlabel("iteration")
plt.ylabel("time (s)")
plt.title("Per-iteration time")
plt.grid(True)
display(plt.gcf())
plt.close()

print(f"{i}  iter_time={duration:.4f}s")
# mean, sum_w, counts = ensemble.finalize()